# Scraping en cascade
- Scraper la page suivante: https://www.charlesbordet.com/fr/blog/
- Récupérer la liste des articles
- Pour chaque articles retrouver l'url vers la page dédiée à cet article (par exemple pour le 1er article l'adresse est : https://www.charlesbordet.com/fr/reactive-shiny/)
- Une fois arrivé sur cette page récupérer le sommaire
- Produire une liste d'articles comprenant 
    - Le titre
    - La description
    - Le tps de lecture
    - (Et surtout) le sommaire

In [27]:
from bs4 import BeautifulSoup
import requests

In [23]:
# Récupération de la liste des articles sous forme d'une liste de liens

page = requests.get('https://www.charlesbordet.com/fr/blog/')
soup = BeautifulSoup(page.content, 'html.parser')
urls = soup.find_all(class_='archive__item-title')
for i in range(len(urls)): urls[i] = 'https://www.charlesbordet.com/fr/' + str(urls[i])[66:].split('"')[0]

In [24]:
# Fonction pour transformer les sommaires en dictionnaires
# Pompé de Stack Overflow, à adapter

def ul_to_dict(ul):
    result = {}
    for li in ul.find_all("li", recursive=False):
        key = next(li.stripped_strings)
        ul = li.find("ul")
        if ul: result[key] = ul_to_dict(ul)
        else: result[key] = None
    return result

In [25]:
# Production d'une liste d'articles sous forme de dictionnaires

articles = []

descriptions = soup.find_all(class_='archive__item-excerpt')
for i in range(len(descriptions)): descriptions[i] = descriptions[i].get_text()[:-1]

for url in urls:
    articles.append(dict())
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    articles[-1]['Titre'] = soup.find(class_='titre-article').get_text()
    articles[-1]['Description'] = descriptions.pop(0)
    articles[-1]['Temps de lecture (minutes)'] = int(soup.find(class_='page__meta').get_text()[8:-20])
    articles[-1]['Sommaire'] = ul_to_dict(soup.find(class_='toc__menu'))

In [26]:
for i in range(len(articles)): print(articles[i]['Sommaire'], '\n')

{'C’est quoi la réactivité\xa0?': None, 'Les fonctions de réactivité\xa0: req, isolate, etc.': {'La fonction reactive()': None, 'La fonction eventReactive()': None, 'La fonction observe()': None, 'La fonction observeEvent()': None, 'La fonction reactiveValues()': None, 'La fonction isolate()': None, 'La fonction req()': None}, 'Mes petites astuces autour de la réactivité': {'Comment forcer la réactivité avec Sys.time()': None, 'Comment accélérer un calcul avec bindCache': None, 'Comment avoir plusieurs inputs dans un observeEvent': None}, "Cas d'usage: Dans un shinydashboard": None, "Cas d'usage: Les filtres interdépendants": {'Solution 1\xa0: Supprimer l’interdépendance': None, 'Solution 2\xa0: Ajouter un bouton': None, 'Solution 3\xa0: Filtres emboîtés avec garantie de changement': None, 'Solution 4 : Filtres emboîtés sans garantie de changement': None, 'Solution 5 : Si les filtres sont rapides à calculer': None}} 

{'Le point de départ': None, 'La solution naïve': None, 'La solution